[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/training/pytorch/convert_pytorch_training_torchnano.ipynb)

# Convert PyTorch Training Loop to Use `TorchNano`

> 📚 **Related Reading**
>
> If you have already defined a PyTorch training loop function with a model, optimizers, and dataloaders as parameters, you could refer to [this guide](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/Training/PyTorch/use_nano_decorator_pytorch_training.html) to use `@nano` decorator, which is a simpler way to gain acceleration from BigDL-Nano.

`TorchNano` API integrates multiple optimizations to accelerate custom PyTorch training loop. As a pure PyTorch user, you could apply few changes to your existing code to use `TorchNano`.

To use `TorchNano`, you need to install BigDL-Nano for PyTorch first:

In [ ]:
!pip install --pre --upgrade bigdl-nano[pytorch] # install the nightly-built version
!source bigdl-nano-init # set environment variables

> 📝 **Note**
>
> Before starting your PyTorch application, it is highly recommended to run `source bigdl-nano-init` to set several environment variables based on your current hardware. Empirically, these variables will bring big performance increase for most PyTorch applications on training workloads.

> ⚠️ **Warning**
> 
> For Jupyter Notebook users, we recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

## PyTorch Training Loops Example

Suppose you would like to finetune a [ResNet-18 model](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) (pretrained on ImageNet dataset) on [OxfordIIITPet dataset](https://pytorch.org/vision/main/generated/torchvision.datasets.OxfordIIITPet.html), you may create datasets, the model and define your training loops as follows:

In [ ]:
# Define model and dataloader

from torch import nn
from torchvision.models import resnet18

class MyPytorchModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        # Here the size of each output sample is set to 37.
        self.model.fc = nn.Linear(num_ftrs, 37)

    def forward(self, x):
        return self.model(x)


import torch
from torchvision import transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data.dataloader import DataLoader

def create_train_dataloader():
    train_transform = transforms.Compose([transforms.Resize(256),
                                          transforms.RandomCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ColorJitter(brightness=.5, hue=.3),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

    # apply data augmentation to the tarin_dataset
    train_dataset = OxfordIIITPet(root="/tmp/data", transform=train_transform, download=True)

    # prepare data loaders
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    return train_dataloader

In [ ]:
from tqdm import tqdm

def train_loops():
    model = MyPytorchModule()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    loss_fuc = torch.nn.CrossEntropyLoss()
    train_loader = create_train_dataloader()

    num_epochs = 5

    for epoch in range(num_epochs):

        model.train()
        train_loss, num = 0, 0
        with tqdm(train_loader, unit="batch") as tepoch:
            for data, target in tepoch:
                tepoch.set_description(f"Epoch {epoch}")
                optimizer.zero_grad()
                output = model(data)
                loss = loss_fuc(output, target)
                loss.backward()
                optimizer.step()
                loss_value = loss.sum()
                train_loss += loss_value
                num += 1
                tepoch.set_postfix(loss=loss_value)
        print(f'Train Epoch: {epoch}, avg_loss: {train_loss / num}')

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; _The definition of_ `MyPytorchModule` _and_ `create_train_dataloader` _can be found in the_ [runnable example](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/training/pytorch/convert_pytorch_training_torchnano.ipynb).

## Convert to `TorchNano`

There are 5 simple steps to convert your PyTorch code to use `TorchNano`:

1. Import `TorchNano`
2. Subclass `TorchNano` and override its `train` method
3. Move the code for your custom training loops inside the `TorchNano`'s `train` method
4. Call `TorchNano`'s `setup` method to set up model, optimizer(s), and dataloader(s) for accelerated training
5. Replace `loss.backward()` with `self.backward(loss)`

In [ ]:
# Step 1. import TorchNano
from bigdl.nano.pytorch import TorchNano

# Step 2. subclass TorchNano and override its train method
class MyNano(TorchNano):
    def train(self):
        # Step 3. Move the code for your custom training loops inside the train method
        model = MyPytorchModule()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
        loss_fuc = torch.nn.CrossEntropyLoss()
        train_loader = create_train_dataloader()

        # Step 4. call setup method to set up model, optimizer(s),
        #         and dataloader(s) for accelerated training
        model, optimizer, train_loader = self.setup(model, optimizer, train_loader)
        num_epochs = 5

        for epoch in range(num_epochs):

            model.train()
            train_loss, num = 0, 0
            with tqdm(train_loader, unit="batch") as tepoch:
                for data, target in tepoch:
                    tepoch.set_description(f"Epoch {epoch}")
                    optimizer.zero_grad()
                    output = model(data)
                    loss = loss_fuc(output, target)
                    # Step 5. Replace loss.backward() with self.backward(loss)
                    self.backward(loss)
                    optimizer.step()
                    loss_value = loss.sum()
                    train_loss += loss_value
                    num += 1
                    tepoch.set_postfix(loss=loss_value)
            print(f'Train Epoch: {epoch}, avg_loss: {train_loss / num}')

> 📝 **Note**
>
> To make sure that the converted `TorchNano` still has a functional training loop, there are some requirements:
>
> - there should be one and only one instance of `torch.nn.Module` as model in the training loop 
> - there should be at least one instance of `torch.optim.Optimizer` as optimizer in the training loop
> - there should be at least one instance of `torch.utils.data.DataLoader` as dataloader in the training loop

You could then do the training by instantiating `MyNano` and calling its `train` method:

In [ ]:
MyNano().train()

> 📝 **Note**
>
> Due to the optimized environment variables set by `source bigdl-nano-init`, you could already experience some training acceleration after converting your PyTorch code to use `TorchNano`.
> 
> For more optimizations provided by `TorchNano`, you can refer to the Related Readings.

> 📚 **Related Readings**
> 
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/install.html)
> - [How to accelerate a PyTorch application on training workloads through Intel® Extension for PyTorch*](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/Training/PyTorch/accelerate_pytorch_training_ipex.html)
> - [How to accelerate a PyTorch application on training workloads through multiple instances](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/Training/PyTorch/accelerate_pytorch_training_multi_instance.html)
> - [How to use the channels last memory format in your PyTorch application for training](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/Training/PyTorch/pytorch_training_channels_last.html)
> - [How to conduct BFloat16 Mixed Precision training in your PyTorch application](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/Training/PyTorch/accelerate_pytorch_training_bf16.html)